In [7]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pickle
import xgboost as xgb

def rgb_to_hex(rgb):
    return f'#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}'

root = tk.Tk()
root.title("Elastic Modulus of Geopolymer Concrete")

color_rgb = (230, 250, 250)
color_hex = rgb_to_hex(color_rgb)

style = ttk.Style()
style.configure('TEntry', fieldbackground='white')
style.configure('ReadOnly.TEntry', fieldbackground='light grey')

main_container = tk.Frame(root, bg=color_hex)
main_container.pack(fill='both', expand=True, padx=10, pady=10)

left_container = tk.Frame(main_container, bg=color_hex)
left_container.pack(side=tk.LEFT, padx=10, pady=10, anchor='n', fill='y')

right_container = tk.Frame(main_container, bg=color_hex)
right_container.pack(side=tk.LEFT, padx=10, fill='both', expand=True)

frame1 = tk.LabelFrame(left_container, text="Please Select an Input Combination", bg=color_hex, padx=10, pady=10)
frame1.pack(fill='both', padx=10, pady=(10, 5), anchor='n')

estimation_frame = tk.LabelFrame(left_container, text="Estimation of Elastic Modulus of Geopolymer Concrete", bg=color_hex, padx=10, pady=10)
estimation_frame.pack(fill='both', padx=10, pady=(5, 10), anchor='n')

inner_frame = tk.LabelFrame(right_container, text="Input Variables", bg=color_hex, padx=10, pady=10)
inner_frame.pack(fill='both', expand=True, padx=10, pady=10)

inner_frame.columnconfigure(2, minsize=8)  # Adds space after the "Value" column
inner_frame.columnconfigure(4, minsize=8)  # Adds space before the "Maximum" column

min_values = {
    "SCM-SiO₂": 27.15, "SCM-Al₂O₃": 9.2, "SCM-Fe₂O₃": 0, "SCM-CaO": 0.2,
    "SCM-MgO": 0, "CA": 660, "FA": 340, "SS-SiO₂": 0,
    "SH": 0, "T-Na₂O": 8.85, "SP": 0, "TW": 82.7,
    "HTCD": 0, "Age": 1, "CS": 2.72
}

max_values = {
    "SCM-SiO₂": 75.66, "SCM-Al₂O₃": 31.9, "SCM-Fe₂O₃": 17.76, "SCM-CaO": 51.8,
    "SCM-MgO": 9.27, "CA": 1298, "FA": 958, "SS-SiO₂": 100.55,
    "SH": 91.71, "T-Na₂O": 105.91, "SP": 22.5, "TW": 268.17,
    "HTCD": 7, "Age": 695, "CS": 104.82
}

options = [
    "SCM-CaO, T-Na₂O, TW, HTCD, and CS",
    "SCM-Fe₂O₃, SCM-CaO, FA, SH, TW, and CS",
    "SCM-Al₂O₃, SCM-MgO, FA, SS-SiO₂, TW, HTCD, and CS",
    "SCM-Al₂O₃, SCM-MgO, FA, SS-SiO₂, TW, HTCD, Age, and CS",
    "SCM-SiO₂, SCM-Al₂O₃, CA, FA, SS-SiO₂, T-Na₂O, TW, Age, and CS",
    "SCM-Al₂O₃, SCM-CaO, CA, FA, SS-SiO₂, SP, TW, HTCD, Age, and CS"
]

option_percentages = {
    options[0]: "18.6%",
    options[1]: "17.4%",
    options[2]: "17.5%",
    options[3]: "17.1%",
    options[4]: "16.2%",
    options[5]: "17.3%"
}

selected_option = tk.StringVar(value=options[0])

materials_units = [
    ("SCM-SiO₂", "%", 48.55), ("SCM-Al₂O₃", "%", 18.33), ("SCM-Fe₂O₃", "%", 5.72), ("SCM-CaO", "%", 17.51),
    ("SCM-MgO", "%", 2.77), ("CA", "kg/m³", 1040.95), ("FA", "kg/m³", 690.26), ("SS-SiO₂", "kg/m³", 38.54),
    ("SH", "kg/m³", 25.41), ("T-Na₂O", "kg/m³", 35.48), ("SP", "kg/m³", 3.87),
    ("TW", "kg/m³", 145.59), ("HTCD", "Days", 0.78), ("Age", "Days", 40.5), ("CS", "MPa", 39.03)
]

entry_widgets = {}
min_entry_widgets = {}
max_entry_widgets = {}

def update_entries():
    prediction_var.set('')  # Add this line to clear the readonly_entry
    selected = selected_option.get().split(", ")
    for material, unit, default_value in materials_units:
        entry = entry_widgets[material]
        min_entry = min_entry_widgets[material]
        max_entry = max_entry_widgets[material]
        
        if material in selected or material == "CS":
            entry.config(state=tk.NORMAL)
            entry.delete(0, tk.END)
            entry.insert(0, str(default_value))
            
            min_entry.config(state='normal')
            min_entry.delete(0, tk.END)
            min_entry.insert(0, str(min_values[material]))
            min_entry.config(state='readonly')
            
            max_entry.config(state='normal')
            max_entry.delete(0, tk.END)
            max_entry.insert(0, str(max_values[material]))
            max_entry.config(state='readonly')
        else:
            entry.delete(0, tk.END)
            entry.config(state=tk.DISABLED)
            
            min_entry.config(state='normal')
            min_entry.delete(0, tk.END)
            min_entry.config(state='readonly')
            
            max_entry.config(state='normal')
            max_entry.delete(0, tk.END)
            max_entry.config(state='readonly')

def compute_action():
    selected_option_text = selected_option.get()
    selected_option_index = options.index(selected_option_text) + 5
    all_numbers = True
    wrong_inputs = False
    new_data = []
    
    for material, unit, default_value in materials_units:
        if material in selected_option_text.split(", ") or material == "CS":
            entry = entry_widgets[material]
            try:
                value = float(entry.get())
                if min_values[material] <= value <= max_values[material]:
                    new_data.append(value)
                else:
                    wrong_inputs = True
            except ValueError:
                all_numbers = False
                break
    
    if all_numbers and not wrong_inputs:
        model_file = f'xgboost_model{selected_option_index}v.pkl'
        with open(model_file, 'rb') as f:
            loaded_model = pickle.load(f)
        predictions = loaded_model.predict([new_data])
        prediction_var.set(f"{predictions[0]:.2f}")
    else:
        messagebox.showerror("Error", "Please ensure all inputs are numerical values and fall within the specified range!")

for option in options:
    rb = tk.Radiobutton(frame1, text=option, variable=selected_option, value=option, command=update_entries, bg=color_hex)
    rb.pack(anchor=tk.W, padx=5, pady=5)

compute_button = tk.Button(estimation_frame, text="Compute", bg='light blue', fg='black', width=10, command=compute_action)
compute_button.pack(side=tk.LEFT, padx=5)

prediction_var = tk.StringVar()
readonly_entry = ttk.Entry(estimation_frame, textvariable=prediction_var, state='readonly', width=10, style='TEntry')
readonly_entry.pack(side=tk.LEFT, padx=(0, 5))

tk.Label(estimation_frame, text="GPa ±", bg=color_hex).pack(side=tk.LEFT)

percentage_label = tk.Label(estimation_frame, text=option_percentages[options[0]], bg=color_hex)
percentage_label.pack(side=tk.LEFT)

refresh_button = tk.Button(estimation_frame, text="Refresh", bg='light blue', fg='black', width=10, command=update_entries)
refresh_button.pack(side=tk.LEFT, padx=5)

credits_label = tk.Label(left_container, text="  Developed by Golafshani et al.", bg=color_hex, font=('TkDefaultFont', 10, 'bold'))
credits_label.pack(anchor='w', pady=(8, 0))

tk.Label(inner_frame, text="Values", bg=color_hex).grid(row=0, column=1, sticky="w")
tk.Label(inner_frame, text="Minimum", bg=color_hex).grid(row=0, column=3, sticky="w")
tk.Label(inner_frame, text="Maximum", bg=color_hex).grid(row=0, column=5, sticky="w")

for i, (material, unit, _) in enumerate(materials_units):
    label = tk.Label(inner_frame, text=f"{material} ({unit}):", bg=color_hex)
    label.grid(row=i + 1, column=0, sticky="w")
    entry = tk.Entry(inner_frame, width=10)
    entry.grid(row=i + 1, column=1)
    entry_widgets[material] = entry
    
    min_entry = ttk.Entry(inner_frame, width=10, style='ReadOnly.TEntry')
    min_entry.grid(row=i + 1, column=3)
    min_entry_widgets[material] = min_entry
    min_entry.config(state='readonly')
    
    max_entry = ttk.Entry(inner_frame, width=10, state='readonly', style='ReadOnly.TEntry')
    max_entry.grid(row=i + 1, column=5)
    max_entry_widgets[material] = max_entry

update_entries()

root.mainloop()
